In [ ]:
import os
import json
import numpy as np

train_dir='/home/ananconda/data/SLRDataSet'


json_foler_path = '/home/anaconda/data/SLRDataSet/train/label/landmark'

for person in os.listdir(json_foler_path):
    for camdir in os.listdir(os.path.join(json_foler_path, person)):
        for filename in os.listdir(os.path.join(json_foler_path, person, camdir)):
            if filename.endswith('.json'):
                file_path = os.path.join(json_foler_path,person,camdir,filename)
                # Open and read the JSON file
                with open(file_path, 'r') as json_file:
                    try:
                        data = json.load(json_file)  # Parse JSON content
                        lh_points= data['people']['hand_left_keypoints_3d']
                        rh_points= data['people']['hand_right_keypoints_3d']
                        p_points= data['people']['pose_keypoints_3d']
                        counter = 0
                        pcounter = 0
                        for j in range(0, len(p_points), 4):
                            xp = p_points[j]     # x-coordinate
                            yp = p_points[j+1]   # y-coordinate
                            zp = p_points[j+2]   # z-coordinate
                            
                            coordp = np.array([xp, yp ,zp ], dtype=np.int64)

                            pcounter += 1

                        for i in range(0, len(lh_points), 4):
                            xl = lh_points[i]     # x-coordinate
                            yl = lh_points[i+1]   # y-coordinate
                            zl = lh_points[i+2]   # z-coordinate
                            # Draw a circle on the image at (x, y)
                            # print(x,'y:',y)
                            coordl = np.array([xl, yl, zl], dtype=np.int64)

                            xr = rh_points[i]     # x-coordinate
                            yr = rh_points[i+1]   # y-coordinate
                            zr = rh_points[i+2]   # z-coordinate
                            
                            coordr = np.array([xr, yr, zr], dtype=np.int64)
                            counter += 1
                    except json.JSONDecodeError as e:
                        print(f"Error reading {filename}: {e}")
                    break


In [20]:
import os
import json
import numpy as np

train_dir='/home/ananconda/data/SLRDataSet'


json_foler_path = '/home/anaconda/data/SLRDataSet/train/label/landmark'

for person in os.listdir(json_foler_path):
    for camdir in os.listdir(os.path.join(json_foler_path, person)):
        for filename in os.listdir(os.path.join(json_foler_path, person, camdir)):
            data = json.load(open(os.path.join(json_foler_path, person, camdir, filename)))
            break
print (data)

{'version': 1.3, 'people': {'person_id': -1, 'face_keypoints_2d': [806.6193022456285, 280.6592752815946, 1, 807.8654866687859, 303.0582458443575, 1, 812.7528838365007, 324.8948603492707, 1, 815.7953580858771, 347.8338900620014, 1, 823.3798256999974, 368.5974657416261, 1, 836.3423081308104, 384.50984338005657, 1, 851.9409471046703, 398.9219929542728, 1, 868.3498504842298, 408.8265899060527, 1, 888.2909630949182, 413.5224830506032, 1, 908.1738880034356, 408.9100742557502, 1, 925.3514554557139, 395.9241967199732, 1, 940.7973661815512, 380.77447671644154, 1, 953.6190610600439, 363.32183287994894, 1, 961.0677010209673, 343.7210105946599, 1, 962.221882595343, 327.878713242763, 1, 965.2181166801529, 307.9219408047569, 1, 967.2953566133195, 287.4996054863882, 1, 819.5568941297824, 272.05305540346154, 1, 828.7983123982754, 265.50598969432343, 1, 840.9362688481425, 265.5396630195599, 1, 854.015170780917, 268.5997189860868, 1, 867.0768486659936, 276.19908374273626, 1, 903.2386717346213, 273.65492